In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import GradScaler, autocast
from transformers import Mask2FormerForUniversalSegmentation
from data_load import EnhancedWildScenesDataset
from tqdm import tqdm
import numpy as np
import os
import logging
from utils.metrics import calculate_miou_train, calculate_pixel_accuracy, calculate_dice_coefficient
from utils.losses import CombinedLoss
from utils.log import setup_logger, save_checkpoint
import torch.nn.functional as F
import math
from models.custom_mask2former import CustomMask2Former



def train_epoch(model, dataloader, criterion, optimizer, scheduler, device, num_classes, scaler):
    model.train()
    total_loss = 0
    total_miou = 0
    total_pixel_acc = 0
    total_dice = 0
    num_batches = 0

    for images, labels in tqdm(dataloader, desc="Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(images)

            if outputs.shape[-2:] != labels.shape[-2:]:
                outputs = F.interpolate(outputs, size=labels.shape[-2:],
                                        mode='bilinear', align_corners=False)

            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        pred = torch.argmax(outputs, dim=1)
        miou = calculate_miou_train(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)
        pixel_acc = calculate_pixel_accuracy(pred.cpu().numpy(), labels.cpu().numpy())
        dice = calculate_dice_coefficient(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)

        if not np.isnan(miou):
            total_miou += miou
            total_pixel_acc += pixel_acc
            total_dice += dice
            num_batches += 1

    return (total_loss / len(dataloader),
            total_miou / num_batches if num_batches > 0 else 0.0,
            total_pixel_acc / num_batches if num_batches > 0 else 0.0,
            total_dice / num_batches if num_batches > 0 else 0.0)


def validate_epoch(model, dataloader, criterion, device, num_classes):
    model.eval()
    total_loss = 0
    total_miou = 0
    total_pixel_acc = 0
    total_dice = 0

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            if outputs.shape[-2:] != labels.shape[-2:]:
                outputs = F.interpolate(outputs, size=labels.shape[-2:],
                                        mode='bilinear', align_corners=False)

            loss = criterion(outputs, labels)

            total_loss += loss.item()
            pred = torch.argmax(outputs, dim=1)
            miou = calculate_miou_train(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)
            pixel_acc = calculate_pixel_accuracy(pred.cpu().numpy(), labels.cpu().numpy())
            dice = calculate_dice_coefficient(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)

            total_miou += miou
            total_pixel_acc += pixel_acc
            total_dice += dice

    num_batches = len(dataloader)
    return (total_loss / num_batches,
            total_miou / num_batches,
            total_pixel_acc / num_batches,
            total_dice / num_batches)


def train(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, save_dir, num_classes):
    best_miou = 0
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    scaler = GradScaler()

    for epoch in range(num_epochs):
        current_epoch = epoch + 1
        logging.info(f"Epoch {current_epoch}/{num_epochs}")

        train_loss, train_miou, train_pixel_acc, train_dice = train_epoch(
            model, train_loader, criterion, optimizer, scheduler, device, num_classes, scaler)
        logging.info(f"Epoch {current_epoch} - Train Loss: {train_loss:.4f}, Train mIoU: {train_miou:.4f}, "
                     f"Train Pixel Acc: {train_pixel_acc:.4f}, Train Dice: {train_dice:.4f}")

        val_loss, val_miou, val_pixel_acc, val_dice = validate_epoch(
            model, val_loader, criterion, device, num_classes)
        logging.info(f"Epoch {current_epoch} - Val Loss: {val_loss:.4f}, Val mIoU: {val_miou:.4f}, "
                     f"Val Pixel Acc: {val_pixel_acc:.4f}, Val Dice: {val_dice:.4f}")

        scheduler.step(val_loss)  # 使用验证损失来调整学习率

        metrics = {
            'miou': val_miou,
            'pixel_acc': val_pixel_acc,
            'dice': val_dice
        }

        if val_miou > best_miou:
            best_miou = val_miou
            best_model_path = os.path.join(save_dir, f'best_model_epoch_{current_epoch}.pth')
            save_checkpoint(model, optimizer, current_epoch, metrics, best_model_path)
            logging.info(f"Epoch {current_epoch} - Best model saved with mIoU: {best_miou:.4f}")

        if current_epoch % 5 == 0:
            checkpoint_path = os.path.join(save_dir, f'checkpoint_epoch_{current_epoch}.pth')
            save_checkpoint(model, optimizer, current_epoch, metrics, checkpoint_path)
            logging.info(f"Epoch {current_epoch} - Checkpoint saved")

        current_lr = optimizer.param_groups[0]['lr']
        logging.info(f"Current learning rate: {current_lr:.6f}")

    logging.info(f"Training completed after {num_epochs} epochs.")
    return best_model_path


if __name__ == "__main__":
    # 创建保存目录
    save_dir = os.path.join('model_checkpoints', 'Mask2Former_0')
    os.makedirs(save_dir, exist_ok=True)

    # 设置日志文件路径
    log_file = os.path.join(save_dir, 'training.log')
    setup_logger(log_file)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logging.info(f"Using device: {device}")

    train_loader = EnhancedWildScenesDataset.get_data_loader('train', batch_size=8)
    val_loader = EnhancedWildScenesDataset.get_data_loader('valid', batch_size=8)

    num_classes = 18  

    model = CustomMask2Former(num_classes=num_classes).to(device)

    criterion = CombinedLoss(weight_focal=1.0, weight_dice=0.5)

    # 只优化未冻结的参数
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, weight_decay=0.01)

    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

    num_epochs = 30
    best_model_path = train(model, train_loader, val_loader, criterion, optimizer, scheduler,
                            num_epochs, device, save_dir, num_classes)

    logging.info("Training and prediction completed!")

Using device: cuda
Epoch 1/30
Training: 100%|██████████████████████████████████████████████████████████████████████| 137/137 [09:30<00:00,  4.16s/it]
Epoch 1 - Train Loss: 2.0077, Train mIoU: 0.5558, Train Pixel Acc: 0.6945, Train Dice: 0.5884
Validating: 100%|██████████████████████████████████████████████████████████████████████| 39/39 [02:19<00:00,  3.58s/it]
Epoch 1 - Val Loss: 0.4109, Val mIoU: 0.6496, Val Pixel Acc: 0.8300, Val Dice: 0.6849
Checkpoint saved: model_checkpoints\Mask2Former_0\best_model_epoch_1.pth
Epoch 1 - Best model saved with mIoU: 0.6496
Current learning rate: 0.000100
Epoch 2/30
Training: 100%|██████████████████████████████████████████████████████████████████████| 137/137 [09:03<00:00,  3.97s/it]
Epoch 2 - Train Loss: 0.4203, Train mIoU: 0.6485, Train Pixel Acc: 0.8300, Train Dice: 0.6832
Validating: 100%|██████████████████████████████████████████████████████████████████████| 39/39 [02:21<00:00,  3.62s/it]
Epoch 2 - Val Loss: 0.3671, Val mIoU: 0.6616, Val Pixel

In [2]:
# 調整 從epoch 30學習率就降低為0了，收斂過快，後期loss浮動沒有明顯下降，mIOU 像素精準度 Dice相關係數也沒有明顯上升
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
from torch.cuda.amp import GradScaler, autocast
from transformers import Mask2FormerForUniversalSegmentation
from data_load_for_mask2former import EnhancedWildScenesDataset
from tqdm import tqdm
import numpy as np
import os
import logging
from utils.metrics import calculate_miou_train, calculate_pixel_accuracy, calculate_dice_coefficient
from utils.losses import CombinedLoss
from utils.log import setup_logger, save_checkpoint
import torch.nn.functional as F
import math
from models.custom_mask2former import CustomMask2Former



def train_epoch(model, dataloader, criterion, optimizer, scheduler, device, num_classes, scaler):
    model.train()
    total_loss = 0
    total_miou = 0
    total_pixel_acc = 0
    total_dice = 0
    num_batches = 0

    for images, labels in tqdm(dataloader, desc="Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(images)

            if outputs.shape[-2:] != labels.shape[-2:]:
                outputs = F.interpolate(outputs, size=labels.shape[-2:],
                                        mode='bilinear', align_corners=False)

            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()

        # 调整梯度裁剪的 max_norm 值為0.5
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)

        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        pred = torch.argmax(outputs, dim=1)
        miou = calculate_miou_train(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)
        pixel_acc = calculate_pixel_accuracy(pred.cpu().numpy(), labels.cpu().numpy())
        dice = calculate_dice_coefficient(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)

        if not np.isnan(miou):
            total_miou += miou
            total_pixel_acc += pixel_acc
            total_dice += dice
            num_batches += 1

    return (total_loss / len(dataloader),
            total_miou / num_batches if num_batches > 0 else 0.0,
            total_pixel_acc / num_batches if num_batches > 0 else 0.0,
            total_dice / num_batches if num_batches > 0 else 0.0)


def validate_epoch(model, dataloader, criterion, device, num_classes):
    model.eval()
    total_loss = 0
    total_miou = 0
    total_pixel_acc = 0
    total_dice = 0

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Validating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            if outputs.shape[-2:] != labels.shape[-2:]:
                outputs = F.interpolate(outputs, size=labels.shape[-2:],
                                        mode='bilinear', align_corners=False)

            loss = criterion(outputs, labels)

            total_loss += loss.item()
            pred = torch.argmax(outputs, dim=1)
            miou = calculate_miou_train(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)
            pixel_acc = calculate_pixel_accuracy(pred.cpu().numpy(), labels.cpu().numpy())
            dice = calculate_dice_coefficient(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)

            total_miou += miou
            total_pixel_acc += pixel_acc
            total_dice += dice

    num_batches = len(dataloader)
    return (total_loss / num_batches,
            total_miou / num_batches,
            total_pixel_acc / num_batches,
            total_dice / num_batches)


def train(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, save_dir, num_classes):
    best_miou = 0
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    scaler = GradScaler()

    for epoch in range(num_epochs):
        current_epoch = epoch + 1
        logging.info(f"Epoch {current_epoch}/{num_epochs}")

        train_loss, train_miou, train_pixel_acc, train_dice = train_epoch(
            model, train_loader, criterion, optimizer, scheduler, device, num_classes, scaler)
        logging.info(f"Epoch {current_epoch} - Train Loss: {train_loss:.4f}, Train mIoU: {train_miou:.4f}, "
                     f"Train Pixel Acc: {train_pixel_acc:.4f}, Train Dice: {train_dice:.4f}")

        val_loss, val_miou, val_pixel_acc, val_dice = validate_epoch(
            model, val_loader, criterion, device, num_classes)
        logging.info(f"Epoch {current_epoch} - Val Loss: {val_loss:.4f}, Val mIoU: {val_miou:.4f}, "
                     f"Val Pixel Acc: {val_pixel_acc:.4f}, Val Dice: {val_dice:.4f}")

        scheduler.step(val_loss)  # 使用验证损失来调整学习率

        metrics = {
            'miou': val_miou,
            'pixel_acc': val_pixel_acc,
            'dice': val_dice
        }

        if val_miou > best_miou:
            best_miou = val_miou
            best_model_path = os.path.join(save_dir, f'best_model_epoch_{current_epoch}.pth')
            save_checkpoint(model, optimizer, current_epoch, metrics, best_model_path)
            logging.info(f"Epoch {current_epoch} - Best model saved with mIoU: {best_miou:.4f}")

        if current_epoch % 5 == 0:
            checkpoint_path = os.path.join(save_dir, f'checkpoint_epoch_{current_epoch}.pth')
            save_checkpoint(model, optimizer, current_epoch, metrics, checkpoint_path)
            logging.info(f"Epoch {current_epoch} - Checkpoint saved")

        current_lr = optimizer.param_groups[0]['lr']
        logging.info(f"Current learning rate: {current_lr:.6f}")

    logging.info(f"Training completed after {num_epochs} epochs.")
    return best_model_path


if __name__ == "__main__":
    # 创建保存目录
    save_dir = os.path.join('model_checkpoints', 'Mask2Former_1')
    os.makedirs(save_dir, exist_ok=True)

    # 设置日志文件路径
    log_file = os.path.join(save_dir, 'training.log')
    setup_logger(log_file)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logging.info(f"Using device: {device}")

    train_loader = EnhancedWildScenesDataset.get_data_loader('train', batch_size=8)
    val_loader = EnhancedWildScenesDataset.get_data_loader('valid', batch_size=8)

    num_classes = 18  

    model = CustomMask2Former(num_classes=num_classes).to(device)

    # 調整損失函數比重
    criterion = CombinedLoss(weight_focal=0.75, weight_dice=0.25)

    # 降低優化器的weight_decay，嘗試不同的學習率
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, weight_decay=0.001)
    
    num_epochs = 30
    
    # scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)
    scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-6)

    best_model_path = train(model, train_loader, val_loader, criterion, optimizer, scheduler,
                            num_epochs, device, save_dir, num_classes)

    logging.info("Training and prediction completed!")

Using device: cuda
Using device: cuda
Epoch 1/30
Epoch 1/30
Training: 100%|██████████████████████████████████████████████████████████████████████| 137/137 [08:46<00:00,  3.85s/it]
Epoch 1 - Train Loss: 1.2457, Train mIoU: 0.5206, Train Pixel Acc: 0.6670, Train Dice: 0.5511
Epoch 1 - Train Loss: 1.2457, Train mIoU: 0.5206, Train Pixel Acc: 0.6670, Train Dice: 0.5511
Validating: 100%|██████████████████████████████████████████████████████████████████████| 39/39 [02:20<00:00,  3.61s/it]
Epoch 1 - Val Loss: 0.3459, Val mIoU: 0.6420, Val Pixel Acc: 0.8189, Val Dice: 0.6771
Epoch 1 - Val Loss: 0.3459, Val mIoU: 0.6420, Val Pixel Acc: 0.8189, Val Dice: 0.6771
E:\Languages\Anaconda3\envs\image_processing\lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is us

In [3]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import os
import logging
from data_load_for_mask2former import EnhancedWildScenesDataset
from models.custom_mask2former import CustomMask2Former
from utils.metrics import calculate_miou_train, calculate_pixel_accuracy, calculate_dice_coefficient
import matplotlib.pyplot as plt

def setup_logger(log_file):
    os.makedirs(os.path.dirname(log_file), exist_ok=True)
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s - %(levelname)s - %(message)s',
                        handlers=[
                            logging.FileHandler(log_file),
                            logging.StreamHandler()
                        ])

def test(model, dataloader, device, num_classes, save_dir):
    model.eval()
    total_miou = 0
    total_pixel_acc = 0
    total_dice = 0
    num_batches = 0
    image_count = 0

    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(dataloader, desc="Testing")):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            if outputs.shape[-2:] != labels.shape[-2:]:
                outputs = F.interpolate(outputs, size=labels.shape[-2:], mode='bilinear', align_corners=False)

            pred = torch.argmax(outputs, dim=1)
            miou = calculate_miou_train(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)
            pixel_acc = calculate_pixel_accuracy(pred.cpu().numpy(), labels.cpu().numpy())
            dice = calculate_dice_coefficient(pred.cpu().numpy(), labels.cpu().numpy(), num_classes)

            if not np.isnan(miou):
                total_miou += miou
                total_pixel_acc += pixel_acc
                total_dice += dice
                num_batches += 1

            # Save ground truth and prediction for all images in the batch
            for j in range(images.shape[0]):
                save_comparison(images[j], labels[j], pred[j], image_count, save_dir)
                image_count += 1

    logging.info(f"Total images processed and saved: {image_count}")

    return (total_miou / num_batches if num_batches > 0 else 0.0,
            total_pixel_acc / num_batches if num_batches > 0 else 0.0,
            total_dice / num_batches if num_batches > 0 else 0.0)

def normalize_image(image):
    """Normalize image tensor to [0, 1] range."""
    image = image.cpu().numpy()
    image = (image - image.min()) / (image.max() - image.min())
    return np.clip(image, 0, 1)

def save_comparison(image, ground_truth, prediction, index, save_dir):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    
    # Normalize and convert image to correct format
    image_np = normalize_image(image)
    image_np = np.transpose(image_np, (1, 2, 0))  # Change from (C, H, W) to (H, W, C)
    
    # Original image
    ax1.imshow(image_np)
    ax1.set_title("Original Image")
    ax1.axis('off')
    
    # Ground truth
    ax2.imshow(ground_truth.cpu().numpy(), cmap='viridis')
    ax2.set_title("Ground Truth")
    ax2.axis('off')
    
    # Prediction
    ax3.imshow(prediction.cpu().numpy(), cmap='viridis')
    ax3.set_title("Prediction")
    ax3.axis('off')
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f'comparison_{index}.png'))
    plt.close()

if __name__ == "__main__":
    save_dir = 'prediction/Mask2Former_1'
    log_file = os.path.join(save_dir, 'testing.log')
    
    os.makedirs(save_dir, exist_ok=True)
    
    setup_logger(log_file)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logging.info(f"Using device: {device}")

    test_loader = EnhancedWildScenesDataset.get_data_loader('test', batch_size=8)

    num_classes = 18

    model = CustomMask2Former(num_classes=num_classes).to(device)
    
    best_model_path = os.path.join('model_checkpoints', 'Mask2Former_1', 'best_model_epoch_26.pth')
    
    if os.path.exists(best_model_path):
        checkpoint = torch.load(best_model_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        logging.info(f"Loaded model from {best_model_path}")
    else:
        logging.error(f"Model file not found: {best_model_path}")
        exit(1)

    miou, pixel_acc, dice = test(model, test_loader, device, num_classes, save_dir)
    
    logging.info(f"Test Results:")
    logging.info(f"Mean IoU: {miou:.4f}")
    logging.info(f"Pixel Accuracy: {pixel_acc:.4f}")
    logging.info(f"Dice Coefficient: {dice:.4f}")

2024-07-19 04:56:58,408 - INFO - Using device: cuda
2024-07-19 04:57:02,521 - INFO - Loaded model from model_checkpoints\Mask2Former_1\best_model_epoch_26.pth
Testing: 100%|█████████████████████████████████████████████████████████████████████████| 19/19 [01:49<00:00,  5.78s/it]
2024-07-19 04:58:52,435 - INFO - Total images processed and saved: 152
2024-07-19 04:58:52,436 - INFO - Test Results:
2024-07-19 04:58:52,438 - INFO - Mean IoU: 0.7283
2024-07-19 04:58:52,439 - INFO - Pixel Accuracy: 0.8633
2024-07-19 04:58:52,440 - INFO - Dice Coefficient: 0.7838
